##### BIBLIOTECAS

In [2]:
import os
import io
import sys
import zipfile
import tempfile
import subprocess

import librosa
import parselmouth

import scipy.signal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import seaborn as sns
import matplotlib.pyplot as plt

print(f'Versão do Cuda: {torch.__version__}')
print(f'Disponibilidade da GPU: {torch.cuda.is_available()}')

Versão do Cuda: 2.9.0+cu130
Disponibilidade da GPU: True


##### EXTRAÇÃO DAS FEATURES

- Fo(Hz): A frequência fundamental, que representa a altura média da voz.
- Fhi(Hz): A frequência fundamental máxima, que indica a altura mais alta do sinal de voz.
- Flo(Hz): A frequência fundamental mínima, que representa a altura mais baixa do sinal de voz.

- Jitter:
    - Jitter(%): Representa a porcentagem de variação de frequência (jitter) na voz, indicando irregularidades na altura.
    - Jitter(Abs): Jitter absoluto, que mede a quantidade bruta de perturbação de frequência.
    - RAP, PPQ, DDP: Métricas de jitter específicas adicionais que ajudam a quantificar as variações de frequência ao longo do tempo, oferecendo diferentes maneiras de calcular irregularidades na altura.

- Shimmer: Mede a variação de amplitude, indicando a flutuação na intensidade do sinal de voz.
- Shimmer(dB): Uma versão do shimmer baseada em decibéis que quantifica as perturbações de amplitude em unidades logarítmicas.
- Shimmer:APQ3, Shimmer:APQ5, MDVP:APQ, Shimmer:DDA: Várias métricas de shimmer que avaliam diferentes aspectos da instabilidade de amplitude, incluindo variações de curto e longo prazo.

- NHR: Quantifica a quantidade de ruído em relação aos componentes harmônicos da voz, fornecendo informações sobre a qualidade vocal. Um NHR mais alto sugere um sinal de voz mais ruidoso.
- HNR: Mede a proporção entre harmônicos (sinal de voz nítido) e ruído. Um valor mais alto indica melhor clareza do sinal de voz e menos ruído.

- RPDE: Entropia da densidade do período de recorrência, uma medida da complexidade e periodicidade do sinal de voz.
- DFA: Análise de flutuação detendenciada, um método usado para analisar a autossimilaridade do sinal de voz, revelando dependências de longo prazo e comportamento fractal.
- spread1, spread2: Medidas de variações na dispersão do sinal, que ajudam a entender como o sinal flutua ao longo do tempo.
- D2: Dimensão de correlação, uma métrica para medir a complexidade e o número de dimensões no sinal. Ela ajuda a quantificar a complexidade dinâmica do sinal de voz.
- PPE: A entropia do período de altura quantifica a irregularidade ou desordem nos períodos de altura, indicando o nível de imprevisibilidade no sinal de voz.

Acho que a melhor forma de fazer isso é dividir em blocos:
- Extração de F0 e estatísticas básicas (Fo, Fhi, Flo)
- Extração de perturbações de frequência e amplitude (Jitter e Shimmer, com todas as variações)
- Extração de medidas não lineares (RPDE, DFA, D2, spread1, spread2, PPE)

A função que usa Praat para as medidas clássicas e librosa + métodos numéricos para as medidas não lineares (RPDE, DFA etc).

In [ ]:
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)

    try:
        pitch = snd.to_pitch(pitch_floor=75.0, pitch_ceiling=600.0)
    except Exception as e:
        print(f"Erro ao calcular Pitch (to_pitch): {e}", file=sys.stderr)
        return np.full(8, np.nan, dtype=np.float64)

    f0_values = np.array(pitch.selected_array['frequency'], dtype=np.float64)
    f0_values = f0_values[f0_values > 0]
    f0_values_filtered = f0_values[(f0_values > 50) & (f0_values < 500)]
    
    if len(f0_values_filtered) == 0:
        f0_mean = f0_max = f0_min = f0_std = np.nan
    else:
        f0_mean = np.mean(f0_values_filtered)
        f0_max  = np.max(f0_values_filtered)
        f0_min  = np.min(f0_values_filtered)
        f0_std  = np.std(f0_values_filtered)

    try:
        pointProcess = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75.0, 600.0)
        jitter_local = parselmouth.praat.call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)

        shimmer_local_percent = parselmouth.praat.call([snd, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        
        hnr = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75.0, 0.1, 1.0)
        hnr_value = parselmouth.praat.call(hnr, "Get mean", 0, 0)
        
        nhr = 10 ** (-hnr_value / 10) if hnr_value > 0 else np.nan

    except Exception as e:
        print(f"Erro na extração Praat (Jitter/Shimmer/HNR): {e}", file=sys.stderr)
        jitter_local = shimmer_local_percent = hnr_value = nhr = np.nan
        
    features = [
        f0_mean,
        f0_max,
        f0_min,
        f0_std,
        jitter_local,
        shimmer_local_percent,
        hnr_value,
        nhr
    ]
    return np.array(features, dtype=np.float64)

In [3]:
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)

    # =========================
    # 1. Pitch (Fo, Fhi, Flo)
    # =========================
    try:
        pitch = snd.to_pitch(pitch_floor=75.0, pitch_ceiling=600.0)
        f0_values = np.array(pitch.selected_array['frequency'], dtype=np.float64)
        f0_values = f0_values[f0_values > 0]

        if len(f0_values) == 0:
            Fo = Fhi = Flo = np.nan
        else:
            Fo = np.mean(f0_values)
            Fhi = np.max(f0_values)
            Flo = np.min(f0_values)

    except Exception as e:
        print(f"Erro ao calcular Pitch: {e}", file=sys.stderr)
        Fo = Fhi = Flo = np.nan

    # ==========================================
    # 2. Jitter, Shimmer, NHR, HNR (via Praat)
    # ==========================================
    try:
        pointProcess = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75.0, 600.0)

        # --- Jitter ---
        jitter_percent = parselmouth.praat.call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3) * 100
        jitter_abs = parselmouth.praat.call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
        rap = parselmouth.praat.call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
        ppq = parselmouth.praat.call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
        ddp = parselmouth.praat.call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)

        # --- Shimmer ---
        shimmer_local = parselmouth.praat.call([snd, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        shimmer_db = parselmouth.praat.call([snd, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        apq3 = parselmouth.praat.call([snd, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        apq5 = parselmouth.praat.call([snd, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        apq11 = parselmouth.praat.call([snd, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        dda = parselmouth.praat.call([snd, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

        # --- Harmonicity / Noise ---
        harmonicity = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75.0, 0.1, 1.0)
        HNR = parselmouth.praat.call(harmonicity, "Get mean", 0, 0)
        NHR = 10 ** (-HNR / 10) if HNR > 0 else np.nan

    except Exception as e:
        print(f"Erro ao calcular Jitter/Shimmer/HNR: {e}", file=sys.stderr)
        jitter_percent = jitter_abs = rap = ppq = ddp = np.nan
        shimmer_local = shimmer_db = apq3 = apq5 = apq11 = dda = np.nan
        HNR = NHR = np.nan

    # =======================================
    # 3. Nonlinear features (RPDE, DFA, D2)
    # =======================================
    try:
        y, sr = librosa.load(audio_path, sr=None)

        # RPDE (Recurrence Period Density Entropy)
        rpde = np.std(np.diff(y)) / np.mean(np.abs(y)) if np.mean(np.abs(y)) != 0 else np.nan

        # DFA (Detrended Fluctuation Analysis)
        def compute_dfa(signal):
            n = len(signal)
            signal = np.cumsum(signal - np.mean(signal))
            scales = np.floor(np.logspace(np.log10(4), np.log10(n / 4), num=20)).astype(int)
            fluct = []
            for s in scales:
                shape = (n // s, s)
                rms = []
                for segment in np.array_split(signal[:shape[0]*shape[1]], shape[0]):
                    coeffs = np.polyfit(range(s), segment, 1)
                    trend = np.polyval(coeffs, range(s))
                    rms.append(np.sqrt(np.mean((segment - trend) ** 2)))
                fluct.append(np.sqrt(np.mean(np.array(rms) ** 2)))
            coeffs = np.polyfit(np.log(scales), np.log(fluct), 1)
            return coeffs[0]

        dfa = compute_dfa(y)

        # Spread1 e Spread2 (estatísticas do espectro)
        S, phase = librosa.magphase(librosa.stft(y))
        centroid = librosa.feature.spectral_centroid(S=S)
        bandwidth = librosa.feature.spectral_bandwidth(S=S)
        spread1 = np.mean(centroid)
        spread2 = np.mean(bandwidth)

        # D2 (Dimensão de correlação simples)
        d2 = np.log(np.var(y)) if np.var(y) > 0 else np.nan

        # PPE (Pitch Period Entropy)
        pitch_values = f0_values[f0_values > 0]
        if len(pitch_values) > 0:
            prob, _ = np.histogram(pitch_values, bins=20, density=True)
            prob = prob[prob > 0]
            ppe = -np.sum(prob * np.log2(prob))
        else:
            ppe = np.nan

    except Exception as e:
        print(f"Erro ao calcular medidas não lineares: {e}", file=sys.stderr)
        rpde = dfa = spread1 = spread2 = d2 = ppe = np.nan

    # =======================
    # 4. Vetor final de saída
    # =======================
    features = [
        Fo, Fhi, Flo,
        jitter_percent, jitter_abs, rap, ppq, ddp,
        shimmer_local, shimmer_db, apq3, apq5, apq11, dda,
        NHR, HNR,
        rpde, dfa, spread1, spread2, d2, ppe
    ]

    return np.array(features, dtype=np.float64)

In [4]:
zip_path = r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\SAND_Challenge_task1_dataset.zip'
target_folder = 'task1/training/rhythmTA'
metadata_path = 'task1/sand_task_1.xlsx'

features_names = [
    "F0_mean_Hz", 
    "F0_max_Hz", 
    "F0_min_Hz", 
    "F0_std_Hz",
    "Jitter_percent", 
    "Shimmer_percent", 
    "HNR_dB", 
    "NHR"
]

data = []

with zipfile.ZipFile(zip_path, 'r') as zipf:
    with zipf.open(metadata_path) as meta_file:
        metadata_df = pd.read_excel(meta_file)
    
    for file in zipf.namelist():
        if file.startswith(target_folder) and file.endswith('.wav'):
            try:
                with zipf.open(file) as audio_file:
                    with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as tmp:
                        tmp.write(audio_file.read())
                        tmp_path = tmp.name

                features = extract_features(tmp_path)

                os.remove(tmp_path)

                filename = os.path.basename(file)
                sample_id = filename.split("_")[0]

                meta_row = metadata_df.loc[metadata_df["ID"] == sample_id]
                if not meta_row.empty:
                    age  = int(meta_row["Age"].values[0])
                    sex  = meta_row["Sex"].values[0]
                    clas = int(meta_row["Class"].values[0])
                else:
                    age = sex = clas = np.nan

                row = [sample_id, age, sex, clas] + list(features)
                data.append(row)

            except Exception as e:
                print(f'Erro no arquivo {file}: {e}', file=sys.stderr)


columns = ['ID', 'Age', 'Sex', 'Class'] + features_names
df = pd.DataFrame(data, columns=columns)
output_csv = r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\features.csv'
df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print('Extração concluída!')

ValueError: 12 columns passed, passed data had 26 columns

##### SETUP

In [ ]:
df = pd.read_csv(r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\features_all.csv')

# X = df.drop(columns=["ID", "Class"])
# y = df["Class"]

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
# X_scaled_df.to_csv(r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\features_normalized_column.csv', index=False)

# X_train, X_test, y_train, y_test = train_test_split(
#     X_scaled, y, test_size=0.2, random_state=42, stratify=y
# )

# rf = RandomForestClassifier(
#     n_estimators=200,
#     max_depth=15,
#     min_samples_leaf=5,
#     max_features='sqrt',
#     random_state=42,
#     n_jobs=-1
# )
# rf.fit(X_train, y_train)

# y_train_pred = rf.predict(X_train)
# y_test_pred = rf.predict(X_test)

# print("Acurácia treino:", accuracy_score(y_train, y_train_pred))
# print("Acurácia teste:", accuracy_score(y_test, y_test_pred))

# print("\nRelatório de classificação (teste):")
# print(classification_report(y_test, y_test_pred))

# print("\nMatriz de confusão (teste):")
# print(confusion_matrix(y_test, y_test_pred))

# importances = pd.Series(rf.feature_importances_, index=X.columns)
# importances.nlargest(20).sort_values().plot(kind="barh", figsize=(8, 6), title="Top 20 Features Importantes")
# plt.show()

# cm = confusion_matrix(y_test, y_test_pred)
# labels = sorted(y_test.unique())

# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
# plt.xlabel("Predito")
# plt.ylabel("Real")
# plt.title("Matriz de Confusão")
# plt.show()

# df = pd.read_csv(r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\features_all.csv')

# cols_to_drop = [col for col in df.columns if col.startswith("NHR")]
# cols_to_drop = [col for col in df.columns if col.startswith("F0_max")]
# cols_to_drop = [col for col in df.columns if col.startswith("F0_min")]
# # cols_to_drop = [col for col in df.columns if col.startswith("Age")]
# # cols_to_drop = [col for col in df.columns if col.startswith("Sex")]

# df = df.drop(columns=cols_to_drop)

# test_ids = [
#     "ID007","ID012","ID021","ID029","ID037","ID046","ID051","ID053","ID054","ID056",
#     "ID063","ID075","ID083","ID090","ID099","ID101","ID105","ID110","ID116","ID122",
#     "ID129","ID136","ID138","ID140","ID158","ID160","ID164","ID171","ID176","ID179",
#     "ID202","ID209","ID227","ID229","ID233","ID245","ID252","ID253","ID260","ID261",
#     "ID263","ID264","ID269","ID270","ID274","ID278","ID284","ID286","ID290","ID302",
#     "ID306","ID323","ID329"
# ]

# train_df = df[~df["ID"].isin(test_ids)].reset_index(drop=True)
# test_df  = df[df["ID"].isin(test_ids)].reset_index(drop=True)

# X_train = train_df.drop(columns=["ID", "Class"])
# y_train = train_df["Class"]

# X_test = test_df.drop(columns=["ID", "Class"])
# y_test = test_df["Class"]

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled  = scaler.transform(X_test)

# rf = RandomForestClassifier(
#     n_estimators=200,
#     max_depth=15,
#     min_samples_leaf=5,
#     max_features='sqrt',
#     random_state=42,
#     n_jobs=-1
# )
# rf.fit(X_train_scaled, y_train)

# y_train_pred = rf.predict(X_train_scaled)
# y_test_pred  = rf.predict(X_test_scaled)

# print("Acurácia treino:", accuracy_score(y_train, y_train_pred))
# print("Acurácia teste:", accuracy_score(y_test, y_test_pred))
# print("\nRelatório de classificação (teste):")
# print(classification_report(y_test, y_test_pred))

# cm = confusion_matrix(y_test, y_test_pred)
# labels = sorted(y_test.unique())

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
# plt.xlabel("Predito")
# plt.ylabel("Real")
# plt.title("Matriz de Confusão")
# plt.show()

# avg_f1_score = f1_score(y_test, y_test_pred, average='macro')
# print("Average F1-score (teste):", avg_f1_score)